In [1]:
import statsmodels.tsa.stattools as ts
import pandas as pd
import statistics
from pandas_datareader import data as pdr
import time
import statsmodels.api as sm
import statistics as stat
from graphframes import *
from pyspark.sql.types import *

#author Mirco A. Mannucci
#author Deborah Tylor


#SP 500 ingestion

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table = data[0]

tickers =table[0].tolist()
tickers = tickers[1:]
#comment out line below to exclude SP 500 Index ticker
tickers.append('^GSPC')

start = '2013-01-01'
end = '2013-12-31'


s=time.time()
d = {}
for ticker in tickers:   
  try:
    d[ticker] = pdr.DataReader(ticker, "yahoo", start=start, end=end)
  except:
    print("An exception occurred downloading ticker " + ticker)
e=time.time()
print('downloaded ticker data in time: ' + str(e-s))

pan = pd.Panel(d)
df = pan.minor_xs("Close")


#last prices 
df_lastprice =df.tail(1)
df.drop(df.tail(1).index,inplace=True)
lst_nodes=[]
for colx in df_lastprice.columns:
    lst_nodes.append([colx, df_lastprice[colx].values[0]])
    
df_nodes = pd.DataFrame(lst_nodes)
df_nodes.columns = ['id', 'lastprice']
df.drop(df.tail(1).index,inplace=True)


def cointegration_test(y, X):    
    X = sm.add_constant(X) # adding a constant
    
    # Step 1: regress one variable on the other 
    ols_result = sm.OLS(y, X).fit() 

    # extract beta_0, beta_1, mean of residual,
    #and standard deviation    
    beta0 = ols_result.params[0]    
    beta1 = ols_result.params[1]
    mean = stat.median( ols_result.resid)
    stdev =stat.stdev( ols_result.resid)
    
    # Step 2: obtain the residual (ols_resuld.resid)
    residuals = ols_result.resid 
    # Step 3: apply Augmented Dickey-Fuller test to see whether 
    # the residual is stationary  
    adf_result = ts.adfuller(residuals)    
    pvalue = adf_result[1]    
    return [beta0, beta1, mean, stdev,  pvalue]


#calculate cointegration to build edges 
#saves entire model and pvalue

res = {}
s=time.time()
for colx in df.columns:    
    for coly in df.columns:
        try:            
            X =df[colx].values
            Y =df[coly].values
            res[colx, coly] = cointegration_test(Y, X)
        except:
            print("An exception occurred calculating coint for " + colx + " " + coly)           
e=time.time()
print("results of cointegration in time: " + str(e-s))

lst_res=[]
for key in res:
    #make undirected edge list
    lst_res.append([key[0],
                    key[1],
                    res[key][0],
                    res[key][1], 
                    res[key][2], 
                    res[key][3], 
                    res[key][4]  
                    ])
df_coint = pd.DataFrame(lst_res)
df_coint.columns = ['src',
                    'dst', 
                    'beta0', 
                    'beta1', 
                    'mean', 
                    'stdev', 
                    'pvalue']
df_coint.head()

#eliminates all edges whose pvalus is not NAN and above threshold 0.05
df_coint = df_coint[df_coint.pvalue.notnull()]
df_coint = df_coint[df_coint.pvalue > 0.05]

#conversion from Panda dataframes to Spark datatrames 

myNodesSchema = StructType([ StructField("id", StringType(), True)\
                            ,StructField("lastprice", FloatType(), True)])
nodes_df = spark.createDataFrame(df_nodes,schema=myNodesSchema)

myEdgesSchema = StructType([ StructField("src", StringType(), True)\
                            ,StructField("dst", StringType(), True)\
                            ,StructField("beta0",  FloatType(), True)\
                            ,StructField("beta1",  FloatType(), True)\
                            ,StructField("mean",   FloatType(), True)\
                            ,StructField("stdev",  FloatType(), True)\
                            ,StructField("pvalue", FloatType(), True)])

edges_df = spark.createDataFrame(df_coint,schema=myEdgesSchema)
cointgraph = GraphFrame(nodes_df,  edges_df)


#last step: PREGEL calculation of alerts (using aggregateMessage() function from GraphFrames)

from pyspark.sql.functions  import *
from graphframes.lib import AggregateMessages as AM

msgToDst = (  (AM.dst["lastprice"] - (AM.edge["beta1"] * AM.src["lastprice"] + AM.edge["beta0"])) -AM.edge["mean"])/AM.edge["stdev"]
agg= cointgraph.aggregateMessages(
  max(AM.msg).alias("alertStatus"),
    sendToDst= msgToDst
)

agg.show()






An exception occurred downloading ticker AMCR
An exception occurred downloading ticker ANET
An exception occurred downloading ticker BBT
An exception occurred downloading ticker BRK.B
An exception occurred downloading ticker BF.B
An exception occurred downloading ticker CF
An exception occurred downloading ticker CFG
An exception occurred downloading ticker CTVA
An exception occurred downloading ticker COTY
An exception occurred downloading ticker DOW
An exception occurred downloading ticker EW
An exception occurred downloading ticker EVRG
An exception occurred downloading ticker FTV
An exception occurred downloading ticker HIG
An exception occurred downloading ticker HPE
An exception occurred downloading ticker INFO
An exception occurred downloading ticker KEYS
An exception occurred downloading ticker KHC
An exception occurred downloading ticker LW
An exception occurred downloading ticker LIN
An exception occurred downloading ticker OKE
An exception occurred downloading ticker PYPL
An exception occurred downloading ticker PNW
An exception occurred downloading ticker PPG
An exception occurred downloading ticker QRVO
An exception occurred downloading ticker ROL
An exception occurred downloading ticker SYF
An exception occurred downloading ticker UA
An exception occurred downloading ticker VMC
An exception occurred downloading ticker WRK
downloaded ticker data in time: 503.0859899520874
/databricks/python/lib/python3.5/site-packages/numpy/linalg/linalg.py:1546: RuntimeWarning: invalid value encountered in greater
 return sum(S > tol)
An exception occurred calculating coint for A ALLE
An exception occurred calculating coint for A HLT
An exception occurred calculating coint for A IQV
An exception occurred calculating coint for A JEF
An exception occurred calculating coint for A NCLH
An exception occurred calculating coint for A NWS
An exception occurred calculating coint for A NWSA
An exception occurred calculating coint for A TWTR
An exception occurred calculating coint for A ZTS
An exception occurred calculating coint for AAL ALLE
An exception occurred calculating coint for AAL HLT
An exception occurred calculating coint for AAL IQV
An exception occurred calculating coint for AAL JEF
An exception occurred calculating coint for AAL NCLH
An exception occurred calculating coint for AAL NWS
An exception occurred calculating coint for AAL NWSA
An exception occurred calculating coint for AAL TWTR
An exception occurred calculating coint for AAL ZTS
An exception occurred calculating coint for AAP ALLE
An exception occurred calculating coint for AAP HLT
An exception occurred calculating coint for AAP IQV
An exception occurred calculating coint for AAP JEF
An exception occurred calculating coint for AAP NCLH
An exception occurred calculating coint for AAP NWS
An exception occurred calculating coint for AAP NWSA
An exception occurred calculating coint for AAP TWTR
An exception occurred calculating coint for AAP ZTS
An exception occurred calculating coint for AAPL ALLE
An exception occurred calculating coint for AAPL HLT
An exception occurred calculating coint for AAPL IQV
An exception occurred calculating coint for AAPL JEF
An exception occurred calculating coint for AAPL NCLH
An exception occurred calculating coint for AAPL NWS
An exception occurred calculating coint for AAPL NWSA
An exception occurred calculating coint for AAPL TWTR
An exception occurred calculating coint for AAPL ZTS
An exception occurred calculating coint for ABBV ALLE
An exception occurred calculating coint for ABBV HLT
An exception occurred calculating coint for ABBV IQV
An exception occurred calculating coint for ABBV JEF
An exception occurred calculating coint for ABBV NCLH
An exception occurred calculating coint for ABBV NWS
An exception occurred calculating coint for ABBV NWSA
An exception occurred calculating coint for ABBV TWTR
An exception occurred calculating coint for ABBV ZTS
/databricks/python/lib/python3.5/site-packages/statsmodels/regression/linear_model